In [1]:
# Chạy khi chưa cài và chạy 1 lần thôi
# Chạy ô này để cài đặt thư viện
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 609.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━ 590.7/899.7 MB 11.1 MB/s eta 0:00:28
ERROR: Operation cancelled by user
^C
Traceback (most recent call last):
  File "/home/ngothang/github/GMP1/server/venv/bin/pip", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/ngothang/github/GMP1/server/venv/lib/python3.12/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ngothang/github/GMP1/server/venv/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 100, in main
    with self.main_context():
  File "/usr/lib/python3.12/contextlib.py", line 144, in __exit__
    next(self.gen)
  File "/home/ngothang/github/GMP1/se

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline  # Thư viện NLP chính
import os

# Thư viện cho Giai đoạn 2 (Machine Learning)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Cài đặt để hiển thị đầy đủ các cột khi in
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# # --- QUAN TRỌNG: HÃY SỬA ĐƯỜNG DẪN NÀY ---
# # Hãy đảm bảo đường dẫn kết thúc bằng dấu '/'
# BASE_DRIVE_PATH = '/content/drive/MyDrive/big_data/data/'
# print(f"Đã kết nối Drive. Thư mục làm việc: {BASE_DRIVE_PATH}")

In [5]:
file_2021= pd.read_csv('data/2021_merged.csv')
file_2022_2023 = pd.read_csv('data/2022-2023_merged.csv')
file_2024_2025 = pd.read_csv('data/2024-2025_merged.csv')

In [7]:
# 1. Đảm bảo 3 file CSV của bạn nằm trong thư mục:

# --- Sửa lại tên file nếu cần ---
file_2021= pd.read_csv('data/2021_merged.csv')
file_2022_2023 = pd.read_csv('data/2022-2023_merged.csv')
file_2024_2025 = pd.read_csv('data/2024-2025_merged.csv')


try:
    # 2. Đọc từng file
    df_2021 = file_2021
    df_2022_2023 = file_2022_2023
    df_2024_2025 = file_2024_2025

    print("Đã đọc thành công 3 file CSV từ Google Drive.")

    # 3. Gộp 3 DataFrame
    df_list = [df_2021, df_2022_2023, df_2024_2025]
    df = pd.concat(df_list, ignore_index=True)

    print(f"Dữ liệu gốc (2021): {len(df_2021)} dòng")
    print(f"Dữ liệu (2022-2023): {len(df_2022_2023)} dòng")
    print(f"Dữ liệu (2024-2025): {len(df_2024_2025)} dòng")
    print(f"==> TỔNG DỮ LIỆU ĐÃ GỘP: {len(df)} dòng")

except FileNotFoundError as e:
    print(f"LỖI: Không tìm thấy file. {e}")
    print("Vui lòng kiểm tra lại ĐƯỜNG DẪN và TÊN FILE trong Drive.")
    raise e

# 4. Chuyển đổi cột 'Date' sang datetime
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# 5. Sắp xếp lại dữ liệu theo thứ tự thời gian (cũ đến mới)
df = df.sort_values(by='Date', ascending=True).reset_index(drop=True)

# 6. Kiểm tra dữ liệu
print("\nThông tin dữ liệu (Data Info) sau khi gộp:")
df.info()
print("\n5 dòng dữ liệu ĐẦU TIÊN (cũ nhất):")
print(df.head())

Đã đọc thành công 3 file CSV từ Google Drive.
Dữ liệu gốc (2021): 9986 dòng
Dữ liệu (2022-2023): 63446 dòng
Dữ liệu (2024-2025): 57515 dòng
==> TỔNG DỮ LIỆU ĐÃ GỘP: 130947 dòng

Thông tin dữ liệu (Data Info) sau khi gộp:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130947 entries, 0 to 130946
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   datepost     130947 non-null  object        
 1   title        130947 non-null  object        
 2   Ticker       130947 non-null  object        
 3   Market       130947 non-null  object        
 4   Date         130947 non-null  datetime64[ns]
 5   Open         130947 non-null  float64       
 6   High         130947 non-null  float64       
 7   Low          130947 non-null  float64       
 8   Close        130947 non-null  float64       
 9   Volume       130947 non-null  int64         
 10  lead1_date   130882 non-null  object        
 11  lead1_close  

In [14]:
# chạy 1 lần thôi vì file đã lưu trong drive
# 1. Tải pipeline phân tích cảm xúc
print("Đang tải mô hình NLP...")
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-xlm-roberta-base-sentiment",
    use_fast=False  # ✅ Tắt fast tokenizer để không bị lỗi
)
print("...Tải mô hình NLP thành công!")

# --- CHỌN 1 TRONG 2 LỰA CHỌN DƯỚI ĐÂY ---

# Lựa chọn A: Chạy thử nghiệm trên 1000 dòng (để test code)
# data_to_process = df.head(1000).copy()

# Lựa chọn B: Chạy trên TOÀN BỘ dữ liệu (Dùng khi chạy thật - SẼ RẤT LÂU)
data_to_process = df.copy()
# ---------------------------------------------

titles_list = data_to_process['title'].tolist()

# 3. Chạy phân tích cảm xúc
print(f"Bắt đầu phân tích cảm xúc cho {len(titles_list)} tiêu đề...")
sentiment_results = sentiment_pipeline(titles_list, batch_size=64, truncation=True)
print("...Hoàn thành phân tích cảm xúc.")

# 4. Gán kết quả trở lại DataFrame
label_map = {'LABEL_0': 'negative', 'LABEL_1': 'neutral', 'LABEL_2': 'positive'}
data_to_process['sentiment_raw'] = [result['label'] for result in sentiment_results]
data_to_process['sentiment'] = data_to_process['sentiment_raw'].map(label_map)

# 5. --- LƯU KẾT QUẢ RA DRIVE ---
#    Đây là bước quan trọng để bạn không phải chạy lại Giai đoạn 1
output_csv_name = 'data_da_gan_nhan_sentiment.csv'
output_csv_path = os.path.join(BASE_DRIVE_PATH, output_csv_name)

print(f"\nĐang lưu kết quả (với cột 'sentiment') ra Drive tại:")
print(output_csv_path)
data_to_process.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
print("...Lưu thành công!")

# 6. Xem kết quả Giai đoạn 1
print("\nPhân bổ các nhãn cảm xúc:")
print(data_to_process['sentiment'].value_counts())

Đang tải mô hình NLP...


ValueError: Converting from SentencePiece and Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast converters: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'Realm

In [ ]:
# chạy 1 lần thôi vì file đã lưu trong drive
# --- Cell 6 ( Gộp, Xử lý và Lưu file) ---
import pandas as pd
import numpy as np
import os

# 1. --- ĐỊNH NGHĨA TÊN FILE ---
TECHNICAL_FILE_NAME = 'data_price.csv'
SENTIMENT_FILE_NAME = 'data_da_gan_nhan_sentiment.csv'
MERGED_OUTPUT_NAME = 'data_merged_final_for_model.csv' # <-- File kết quả
# ----------------------------------------------------

# 2. Đặt đường dẫn
sentiment_csv_path = os.path.join(BASE_DRIVE_PATH, SENTIMENT_FILE_NAME)
technical_csv_path = os.path.join(BASE_DRIVE_PATH, TECHNICAL_FILE_NAME)
merged_output_path = os.path.join(BASE_DRIVE_PATH, MERGED_OUTPUT_NAME)

try:
    # 3. Đọc 2 file dữ liệu
    print(f"Đang đọc file sentiment (BÊN TRÁI) từ: {sentiment_csv_path}")
    df_sentiment = pd.read_csv(sentiment_csv_path)
    print("...Đọc file sentiment thành công.")

    print(f"Đang đọc file kỹ thuật (BÊN PHẢI) từ: {technical_csv_path}")
    df_price = pd.read_csv(technical_csv_path)
    print("...Đọc file kỹ thuật thành công.")

except FileNotFoundError as e:
    print(f"LỖI: Không tìm thấy file. {e}")
    raise

# 4. --- CHUẨN BỊ FILE SENTIMENT (TRÁI) ---
print("\nChuẩn bị file Sentiment...")
if 'datepost' in df_sentiment.columns:
    print("  Đã tìm thấy cột 'datepost'. Sẽ xóa (drop) cột này.")
    df_sentiment = df_sentiment.drop(columns=['datepost'])
if 'Date' in df_sentiment.columns and 'date' not in df_sentiment.columns:
    df_sentiment.rename(columns={'Date': 'date'}, inplace=True)
df_sentiment['date'] = pd.to_datetime(df_sentiment['date']).dt.normalize()
df_sentiment['Ticker'] = df_sentiment['Ticker'].astype(str).str.strip().str.upper()
print("...Chuẩn bị file Sentiment xong.")

# 5. --- CHUẨN BỊ FILE KỸ THUẬT (PHẢI) ---
print("\nChuẩn bị file Kỹ thuật...")
technical_indicators_cols = [
    'ma20', 'ma50', 'ema12', 'ema26', 'macd', 'macd_hist', 'macd_signal',
    'bb_lower', 'bb_mid', 'bb_upper', 'BBB_20_2.0', 'BBP_20_2.0',
    'atr14', 'rsi14', 'stoch_k', 'stoch_d', 'obv', 'vol_ma20'
]
if 'Date' in df_price.columns and 'date' not in df_price.columns:
    df_price.rename(columns={'Date': 'date'}, inplace=True)
if 'Ticker' in df_price.columns and 'ticker' not in df_price.columns:
     df_price.rename(columns={'Ticker': 'ticker'}, inplace=True)
if 'ticker' in df_price.columns:
    df_price.rename(columns={'ticker': 'Ticker'}, inplace=True)
cols_to_get_from_price = ['date', 'Ticker'] + [col for col in technical_indicators_cols if col in df_price.columns]
df_price_clean = df_price[cols_to_get_from_price]
print(f"  Các cột sẽ lấy từ file kỹ thuật: {cols_to_get_from_price[2:]}")
df_price_clean['date'] = pd.to_datetime(df_price_clean['date']).dt.normalize()
df_price_clean['Ticker'] = df_price_clean['Ticker'].astype(str).str.strip().str.upper()
print("...Chuẩn bị file Kỹ thuật xong.")

# 6. GỘP (MERGE) HAI DATAFRAME
print("\nĐang gộp (Left Join) file sentiment và file kỹ thuật...")
final_data_merged = pd.merge(
    df_sentiment,
    df_price_clean,
    on=['date', 'Ticker'],
    how='left'
)
print("...Gộp thành công!")

# 7. TẠO FEATURES MỚI (Sentiment Score và Target)
print("\nĐang tạo features và target...")
sentiment_score_map = {'positive': 1, 'negative': -1, 'neutral': 0}
final_data_merged['sentiment_raw_cleaned'] = final_data_merged['sentiment_raw'].astype(str).str.strip()
final_data_merged['sentiment_score'] = final_data_merged['sentiment_raw_cleaned'].map(sentiment_score_map)
final_data_merged['price_diff'] = final_data_merged['lead1_close'] - final_data_merged['Close']
print("...Tạo features và target thành công.")

# 8. LỌC CỘT VÀ LÀM SẠCH (DROPNA)
print("Đang làm sạch dữ liệu (dropna)...")
numeric_features = (
    ['Volume', 'sentiment_score', 'Open', 'High', 'Low', 'Close'] +
    [col for col in technical_indicators_cols if col in final_data_merged.columns]
)
target = 'price_diff'
# Giữ lại các cột khóa để sắp xếp
cols_to_check = ['date', 'Ticker'] + numeric_features + [target]
final_data_cleaned = final_data_merged.dropna(subset=cols_to_check)
print(f"Số dòng dữ liệu còn lại SAU KHI bỏ NaN: {len(final_data_cleaned)}")

# 9. LƯU FILE ĐÃ GỘP VÀ SẠCH RA DRIVE
print(f"\nĐang lưu file đã gộp và làm sạch ra Drive tại: {merged_output_path}")
final_data_cleaned.to_csv(merged_output_path, index=False, encoding='utf-8-sig')
print("...Lưu file thành công! Bạn có thể chạy Cell 7.")

In [ ]:
# chạy 1 lần thôi vì file đã lưu trong drive
# --- Cell 7 (Lọc Cột, Xử lý Outlier) ---
import pandas as pd
import numpy as np
import os

# 1. Tên file đầu vào (từ Cell 6)
MERGED_INPUT_NAME = 'data_merged_final_for_model.csv'
# 2. Tên file đầu ra (đã xử lý)
CLEANED_OUTPUT_NAME = 'data_cleaned_no_outliers.csv'

input_path = os.path.join(BASE_DRIVE_PATH, MERGED_INPUT_NAME)
output_path = os.path.join(BASE_DRIVE_PATH, CLEANED_OUTPUT_NAME)

try:
    # 3. Đọc file
    print(f"Đang đọc file đã gộp từ: {input_path}")
    df = pd.read_csv(input_path)
    print(f"...Đọc thành công {len(df)} dòng.")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {MERGED_INPUT_NAME}.")
    print("Vui lòng chạy lại Cell 6 để tạo file này trước.")
    raise

# 4. --- LỌC CỘT (THEO YÊU CẦU CỦA BẠN) ---
cols_to_keep = [
    'Ticker', 'Market', 'date', 'Open', 'High', 'Low', 'Close', 'Volume',
    'prev5_date', 'ma20', 'ema26', 'macd', 'macd_hist', 'macd_signal',
    'rsi14', 'vol_ma20', 'sentiment_score', 'lead1_close'
]
existing_cols_to_keep = [col for col in cols_to_keep if col in df.columns]
print(f"\nĐã lọc, giữ lại {len(existing_cols_to_keep)} cột.")
df_filtered = df[existing_cols_to_keep].copy()

# 5. --- XỬ LÝ OUTLIER (Sử dụng IQR) ---
numeric_cols_for_outlier = [
    'Open', 'High', 'Low', 'Close', 'Volume', 'ma20', 'ema26', 'macd',
    'macd_hist', 'macd_signal', 'rsi14', 'vol_ma20', 'sentiment_score'
]
numeric_cols_for_outlier = [col for col in numeric_cols_for_outlier if col in df_filtered.columns]

print("Đang xử lý Outlier (IQR)...")
df_no_outlier = df_filtered.copy()
initial_rows = len(df_no_outlier)

for col in numeric_cols_for_outlier:
    Q1 = df_no_outlier[col].quantile(0.25)
    Q3 = df_no_outlier[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_no_outlier = df_no_outlier[
        (df_no_outlier[col] >= lower_bound) & (df_no_outlier[col] <= upper_bound)
    ]

print(f"Số dòng ban đầu: {initial_rows}")
print(f"Số dòng sau khi loại bỏ Outlier: {len(df_no_outlier)}")
df_processed = df_no_outlier.copy()

# 6. --- LƯU FILE ĐÃ XỬ LÝ RA DRIVE ---
print(f"\nĐang lưu file đã xử lý outlier ra Drive tại: {output_path}")
df_processed.to_csv(output_path, index=False, encoding='utf-8-sig')
print("...Lưu file thành công! Bạn có thể chạy Cell 8.")

In [ ]:
# --- Chia và Lưu Train/Test ---
import pandas as pd
import numpy as np
import os

# 1. Tên file đã xử lý (từ Cell 7)
CLEANED_INPUT_NAME = 'data_cleaned_no_outliers.csv'
cleaned_path = os.path.join(BASE_DRIVE_PATH, CLEANED_INPUT_NAME)

try:
    # 2. Đọc file
    print(f"Đang đọc file đã xử lý outlier từ: {cleaned_path}")
    df_processed = pd.read_csv(cleaned_path)
    print(f"...Đọc thành công {len(df_processed)} dòng.")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {CLEANED_INPUT_NAME}.")
    print("Vui lòng chạy lại Cell 7 để tạo file này trước.")
    raise

# 3. --- QUAY LẠI TARGET GỐC: 'lead1_close' ---
target = 'lead1_close'
print(f"\nĐã đặt target là: {target}")
# -----------------------------------------------

# 4. --- ĐỊNH NGHĨA FEATURES ("BẪY") ---
categorical_features = ['Market']
numeric_features = [
    'Open', 'High', 'Low', 'Close',
    'Volume', 'ma20', 'ema26', 'macd',
    'macd_hist', 'macd_signal', 'rsi14', 'vol_ma20', 'sentiment_score'
]
# Lọc lại
numeric_features = [col for col in numeric_features if col in df_processed.columns]
categorical_features = [col for col in categorical_features if col in df_processed.columns]
features_to_use = numeric_features + categorical_features

print(f"Features sẽ dùng ({len(features_to_use)}): {features_to_use}")
print(f"Target sẽ dùng: {target}")

# 5. --- CHIA TRAIN/TEST (THEO THỜI GIAN) ---
df_processed['date'] = pd.to_datetime(df_processed['date'])
df_processed = df_processed.sort_values(by='date').reset_index(drop=True)

# Lấy X và y MỘT LẦN NỮA sau khi sắp xếp
X = df_processed[features_to_use]
y = df_processed[target] # <-- y bây giờ là 'lead1_close'

# Tỉ lệ 70/30
test_percent = 0.3
test_point = int(len(X) * (1 - test_percent))

if test_point == 0 or len(X) == 0:
    print("LỖI: Không đủ dữ liệu để chia train/test.")
else:
    X_train = X.iloc[:test_point] # 70% dữ liệu đầu (cũ nhất)
    X_test = X.iloc[test_point:]  # 30% dữ liệu cuối (mới nhất)
    y_train = y.iloc[:test_point]
    y_test = y.iloc[test_point:]

    print(f"\nKích thước tập Train (70%): {len(X_train)} dòng")
    print(f"Kích thước tập Test (30%): {len(X_test)} dòng")

    # 6. --- (PHẦN MỚI) LƯU TẬP TRAIN VÀ TEST RA FILE ---
    print("\nĐang lưu 2 tập Train/Test ra file (để bạn chạy test lại)...")

    # 6.1. Gộp X_train và y_train
    # (Chúng ta cần cả các cột 'date' và 'Ticker' để tham chiếu)
    train_indices = X_train.index
    train_df_to_save = df_processed.loc[train_indices].copy()

    # 6.2. Gộp X_test và y_test
    test_indices = X_test.index
    test_df_to_save = df_processed.loc[test_indices].copy()

    # 6.3. Định nghĩa đường dẫn
    train_output_path = os.path.join(BASE_DRIVE_PATH, 'train_data.csv')
    test_output_path = os.path.join(BASE_DRIVE_PATH, 'test_data.csv')

    # 6.4. Lưu file
    train_df_to_save.to_csv(train_output_path, index=False, encoding='utf-8-sig')
    test_df_to_save.to_csv(test_output_path, index=False, encoding='utf-8-sig')

    print(f"...Đã lưu file Train tại: {train_output_path}")
    print(f"...Đã lưu file Test tại: {test_output_path}")
    # --- (KẾT THÚC PHẦN MỚI) ---

In [ ]:
# --- So sánh mô hình, thêm Ridge/Lasso ---
import pandas as pd
import numpy as np
import time
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# --- Thư viện các mô hình để so sánh ---
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 1. Tạo 'preprocessor' (Scaling + OneHotEncoding)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)
print("Đã tạo Preprocessor (Scaling + OneHotEncoding).")

# 2. Định nghĩa các mô hình
#    (Thêm Ridge và Lasso theo yêu cầu của bạn)
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (Chống Overfit)': Ridge(alpha=1.0, random_state=42),
    'Lasso (Chống Overfit)': Lasso(alpha=0.1, random_state=42, max_iter=2000),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# 3. Huấn luyện và đánh giá hàng loạt
print(f"Bắt đầu huấn luyện và so sánh {len(models)} mô hình (trên dữ liệu SẠCH)...")
results_list = []

for name, model in models.items():
    print(f"\nĐang huấn luyện: {name}...")

    clf = Pipeline(steps=[('preprocessor', preprocessor),
                         ('model', model)])

    start_time = time.time()
    clf.fit(X_train, y_train)
    train_time = time.time() - start_time

    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)

    metrics = {
        'Model': name,
        'Test R2': r2_score(y_test, y_pred_test),
        'Test MAE': mean_absolute_error(y_test, y_pred_test),
        'Train R2': r2_score(y_train, y_pred_train),
        'Train MAE': mean_absolute_error(y_train, y_pred_train),
        'Trained Pipeline': clf
    }
    results_list.append(metrics)
    print(f"...Hoàn thành {name} trong {train_time:.2f} giây.")

# 4. --- TẠO BẢNG TỔNG HỢP KẾT QUẢ ---
print("\n--- BẢNG TỔNG HỢP KẾT QUẢ (Mô hình SẠCH) ---")
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by='Test MAE', ascending=True)

print(results_df.drop(columns=['Trained Pipeline']).to_markdown(index=False, floatfmt=".4f"))

# 5. LƯU BẢNG KẾT QUẢ
results_table_path = os.path.join(BASE_DRIVE_PATH, 'ket_qua_so_sanh_models_SACH.csv')
print(f"\nĐang lưu bảng kết quả ra file: {results_table_path}")
results_df.drop(columns=['Trained Pipeline']).to_csv(results_table_path, index=False, encoding='utf-8-sig')

# 6. Chọn ra mô hình tốt nhất để chẩn đoán
best_model_pipeline = results_df.iloc[0]['Trained Pipeline']
best_model_name = results_df.iloc[0]['Model']
print(f"\nĐã chọn mô hình tốt nhất ({best_model_name})")

In [ ]:
# ---  Định nghĩa các công cụ Chẩn đoán ---
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, learning_curve, cross_validate
import statsmodels.api as sm

print("Đã import các thư viện chẩn đoán: TimeSeriesSplit, learning_curve, cross_validate")
print("Định nghĩa các hàm: run_time_series_cv, plot_learning_curve, plot_residuals")

# 1. CÔNG CỤ: K-FOLD CROSS-VALIDATION (dành cho Time Series)
def run_time_series_cv(pipeline, X, y, n_splits=5):
    """Chạy Cross-Validation dùng TimeSeriesSplit và in kết quả."""
    print(f"\n--- 1. Chẩn đoán: Time Series Cross-Validation ({n_splits}-Fold) ---")

    cv_splitter = TimeSeriesSplit(n_splits=n_splits)
    scoring_metrics = ['r2', 'neg_mean_absolute_error']

    print("Đang chạy CV...")

    # (Đã đổi 'return_train_scores' thành 'return_train_score')
    scores = cross_validate(pipeline, X, y, cv=cv_splitter,
                            scoring=scoring_metrics, n_jobs=-1,
                            return_train_score=True) # <-- SỬA LỖI Ở ĐÂY

    print("...CV hoàn thành.")

    # Xử lý kết quả
    mean_test_r2 = np.mean(scores['test_r2'])
    mean_test_mae = -np.mean(scores['test_neg_mean_absolute_error'])
    mean_train_r2 = np.mean(scores['train_r2'])
    mean_train_mae = -np.mean(scores['train_neg_mean_absolute_error'])

    print(f"  Kết quả CV trung bình trên {n_splits} fold:")
    print(f"  - Test R2 (Trung bình): {mean_test_r2:.4f}")
    print(f"  - Test MAE (Trung bình): {mean_test_mae:.4f}")
    print(f"  - Train R2 (Trung bình): {mean_train_r2:.4f}")
    print(f"  - Train MAE (Trung..."")")

    if mean_train_mae < (mean_test_mae * 0.7): # Nếu train tốt hơn test 30%
        print("  => KẾT LUẬN: CÓ DẤU HIỆU OVERFITTING (Train tốt hơn Test đáng kể).")
    else:
        print("  => KẾT LUẬN: Overfitting ở mức chấp nhận được.")
    print("--------------------------------------------------")

# 2. CÔNG CỤ: LEARNING CURVE
def plot_learning_curve(pipeline, X, y, n_splits=5):
    """Vẽ Learning Curve để chẩn đoán Overfitting/Underfitting."""
    print("\n--- 2. Chẩn đoán: Vẽ Learning Curve ---")

    cv_splitter = TimeSeriesSplit(n_splits=n_splits)
    train_sizes, train_scores, test_scores = learning_curve(
        pipeline, X, y, cv=cv_splitter,
        scoring='neg_mean_absolute_error', # Dùng MAE
        n_jobs=-1,
        train_sizes=np.linspace(0.2, 1.0, 5) # 5 bước (20% -> 100% data)
    )

    train_scores_mean = -np.mean(train_scores, axis=1)
    test_scores_mean = -np.mean(test_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.grid()
    plt.title("Learning Curve (Đường cong học tập)")
    plt.xlabel("Kích thước tập Huấn luyện (Training examples)")
    plt.ylabel("Lỗi MAE")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Lỗi Training")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Lỗi Validation (CV)")
    plt.legend(loc="best")
    plt.ylim(bottom=0)
    plt.show()

    if train_scores_mean[-1] < (test_scores_mean[-1] * 0.7):
        print("  => KẾT LUẬN: Khoảng cách giữa 2 đường lớn -> Dấu hiệu Overfitting (High Variance).")
    elif test_scores_mean[-1] > (train_scores_mean[-1] * 1.5):
         print("  => KẾT LUẬN: Cả 2 đường đều hội tụ ở mức lỗi cao -> Dấu hiệu Underfitting (High Bias).")
    else:
         print("  => KẾT LUẬN: Hai đường hội tụ tốt.")
    print("--------------------------------------------------")


# 3. CÔNG CỤ: RESIDUAL PLOT
def plot_residuals(y_true, y_pred, model_name):
    """Vẽ biểu đồ Residual (Phần dư) để chẩn đoán Bias."""
    print("\n--- 3. Chẩn đoán: Vẽ Residual Plot ---")

    residuals = y_true - y_pred

    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=y_pred, y=residuals, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel("Giá trị Dự đoán (Predicted Values)")
    plt.ylabel("Phần dư (Residuals = True - Predicted)")
    plt.title(f"Residual Plot - Mô hình {model_name}")
    plt.grid(True)
    plt.show()

    mean_residual = np.mean(residuals)
    print(f"  Residuals (phần dư) trung bình: {mean_residual:.4f}")
    if abs(mean_residual) > 0.1:
         print("  => KẾT LUẬN: Residuals không tập trung quanh 0 -> Dấu hiệu mô hình bị 'Thiên kiến' (Bias).")
    else:
         print("  => KẾT LUẬN: Residuals phân bổ tốt quanh 0.")
    print("--------------------------------------------------")

In [ ]:
# ---  Chạy chẩn đoán ---
print(f"=== BẮT ĐẦU CHẨN ĐOÁN TOÀN DIỆN MÔ HÌNH: {best_model_name} ===")
# 1. Chạy Time Series K-Fold CV

#    (Chạy trên toàn bộ dữ liệu X, y để có đánh giá CV tin cậy nhất)
run_time_series_cv(best_model_pipeline, X, y, n_splits=5)
# 2. Chạy Learning Curve
plot_learning_curve(best_model_pipeline, X, y, n_splits=5)

# 3. Chạy Residual Plot

#    (Chạy trên tập Test để xem hiệu suất trên dữ liệu mới)

print("\n...Đang tạo dự đoán trên tập Test để vẽ Residual Plot...")
y_pred_test_for_residuals = best_model_pipeline.predict(X_test)
plot_residuals(y_test, y_pred_test_for_residuals, best_model_name)
print("\n=== HOÀN THÀNH CHẨN ĐOÁN ===")

In [ ]:
# ---  Biểu đồ kết quả cuối cùng ---

print(f"Đang vẽ biểu đồ Scatter cuối cùng cho mô hình: {best_model_name}...")

# Lấy lại dự đoán từ Cell 11 (hoặc dự đoán lại)

y_pred = best_model_pipeline.predict(X_test)

plot_df = pd.DataFrame({'Chênh lệch Thực tế (y_test)': y_test,

                          'Chênh lệch Dự đoán (y_pred)': y_pred})

plt.figure(figsize=(10, 10))

sns.scatterplot(data=plot_df,

                x='Chênh lệch Thực tế (y_test)',

                y='Chênh lệch Dự đoán (y_pred)',

                alpha=0.5,

                label=f"Dự đoán của ({best_model_name})")

min_val = min(y_test.min(), y_pred.min())

max_val = max(y_test.max(), y_pred.max())

plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Dự đoán Hoàn hảo (y = x)')

plt.title(f'Biểu đồ Scatter (SAU KHI FIX LEAKAGE): Mô hình {best_model_name}')

plt.xlabel('Chênh lệch Thực tế (y_test)')

plt.ylabel('Chênh lệch Dự đoán (y_pred)')

plt.legend()

plt.grid(True)

plt.axis('equal')

plot_path = os.path.join(BASE_DRIVE_PATH, f'bieu_do_best_model_SACH.png')

plt.savefig(plot_path, dpi=300, bbox_inches='tight')

plt.show()

print(f"...Đã lưu biểu đồ cuối cùng tại: {plot_path}")

In [ ]:
# --- "Đóng gói" Model đã huấn luyện---
import joblib
print(f"--- Bắt đầu đóng gói (lưu) mô hình tốt nhất ---")
try:
    # 1. Lấy mô hình TỐT NHẤT (đã được huấn luyện trên 70% X_train)
    #    (Biến 'best_model_pipeline' được tạo ở Cell 9)
    model_to_save = best_model_pipeline
    model_name_to_save = best_model_name

    print(f"Mô hình được chọn để lưu: {model_name_to_save}")
    print("Mô hình này đã được huấn luyện trên 70% dữ liệu (Tập Train).")

except NameError as e:
    print("LỖI: Không tìm thấy 'best_model_pipeline'.")
    print("Vui lòng chạy lại Cell 9 trước khi chạy ô này.")
    raise

# 2. KHÔNG HUẤN LUYỆN LẠI (Theo yêu cầu của bạn)
#    (Chúng ta bỏ qua bước .fit(X_full, y_full))

# 3. LƯU MODEL RA FILE
model_output_path = os.path.join(BASE_DRIVE_PATH, 'stock_predictor_model.joblib')
print(f"\nĐang lưu mô hình đã huấn luyện ra file: {model_output_path}")
joblib.dump(model_to_save, model_output_path)
print("...LƯU MODEL THÀNH CÔNG!")